In [157]:
# imports
import numpy as np
import pandas as pd
from tensorflow import keras
from datetime import datetime
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the TensorBoard notebook extension
import tensorflow as tf
import os
%load_ext tensorboard.notebook

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook


In [158]:
df = pd.read_csv("../../pre_processing/datasets/no_filtered_analysis.csv", sep=",")

In [159]:
df.head()

,RA,EVADIDO,RESID_ARARAS,BOLSISTA,GRADE_CORRENTE,NOTA_MEDIA,PONTUACAO_PS,TOT_MAT_CURSADAS,TOT_APROVACOES,TOT_REPROVACOES,...,SIF029,SIF030,SIF038,SIF039,SIF070,NCS037,NCS040,SIF006,SIF072,SIF073
0,002edf3eae2ce5635d797b35345ffdea,0,1,1,5,7.0,47.0,10,9,1,...,4,0,0,0,0,0,0,0,1,1
1,003719ff8ca88d5266775000fbd3c7f9,1,0,1,4,5.5,47.0,18,15,3,...,0,0,0,0,0,0,0,1,0,0
2,01563df7114a12af3f8cbd0cb4082491,1,0,0,3,3.0,46.0,12,6,6,...,0,0,0,0,0,0,0,4,0,0
3,016ed164e805bc35f38559a5de19f818,1,0,1,3,5.1,47.0,49,37,12,...,1,1,5,4,1,1,1,1,0,0
4,017a04be49784c0140a9506efe4aea86,0,0,1,5,5.4,30.0,10,8,2,...,0,0,0,0,0,0,0,0,2,1


In [160]:
# Dropo coluna RA se existir
if "RA" in df.columns:
    print("Removing column RA")
    df.drop('RA', inplace=True, axis=1)

Removing column RA


In [161]:
X = df.iloc[:, df.columns != "EVADIDO"].values

In [162]:
y = df["EVADIDO"]

In [163]:
y = to_categorical(y)

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [165]:
scaler_object = MinMaxScaler()

In [166]:
scaler_object.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [167]:
scaled_X_train = scaler_object.transform(X_train)

In [168]:
scaled_X_test = scaler_object.transform(X_test)

In [169]:
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential

In [173]:
# Configurate neural network features (layers, layer density, etc)
model = Sequential()
model.add(Dense(units=8, input_dim=61, activation='relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units=8, input_dim=61, activation='relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units=2, activation='softmax'))

adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer = adam, metrics=['accuracy'])

logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

In [174]:
# NN summary
model.summary()
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True) not running on windows

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_100 (Dense)            (None, 8)                 496       
_________________________________________________________________
dropout_43 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_101 (Dense)            (None, 8)                 72        
_________________________________________________________________
dropout_44 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_102 (Dense)            (None, 2)                 18        
Total params: 586
Trainable params: 586
Non-trainable params: 0
_________________________________________________________________


In [175]:
model.fit(scaled_X_train, y_train, validation_data=(scaled_X_test, y_test), epochs=100,
                 callbacks=[tensorboard_callback])

Train on 337 samples, validate on 167 samples
Epoch 1/100
337/337 [==============================] - 1s 2ms/sample - loss: 0.6885 - acc: 0.5015 - val_loss: 0.6839 - val_acc: 0.6048
Epoch 2/100
337/337 [==============================] - 0s 80us/sample - loss: 0.6891 - acc: 0.5579 - val_loss: 0.6774 - val_acc: 0.6886
Epoch 3/100
337/337 [==============================] - 0s 80us/sample - loss: 0.6746 - acc: 0.6706 - val_loss: 0.6719 - val_acc: 0.7545
Epoch 4/100
337/337 [==============================] - 0s 83us/sample - loss: 0.6714 - acc: 0.6855 - val_loss: 0.6661 - val_acc: 0.7725
Epoch 5/100
337/337 [==============================] - 0s 80us/sample - loss: 0.6690 - acc: 0.7033 - val_loss: 0.6599 - val_acc: 0.7725
Epoch 6/100
337/337 [==============================] - 0s 80us/sample - loss: 0.6559 - acc: 0.7448 - val_loss: 0.6536 - val_acc: 0.7784
Epoch 7/100
337/337 [==============================] - 0s 83us/sample - loss: 0.6444 - acc: 0.7507 - val_loss: 0.6480 - val_acc: 0.7485
Epo

337/337 [==============================] - 0s 83us/sample - loss: 0.2498 - acc: 0.9080 - val_loss: 0.2881 - val_acc: 0.8922
Epoch 61/100
337/337 [==============================] - 0s 83us/sample - loss: 0.2827 - acc: 0.9021 - val_loss: 0.2788 - val_acc: 0.8862
Epoch 62/100
337/337 [==============================] - 0s 83us/sample - loss: 0.2735 - acc: 0.9050 - val_loss: 0.2764 - val_acc: 0.8862
Epoch 63/100
337/337 [==============================] - 0s 80us/sample - loss: 0.2780 - acc: 0.8902 - val_loss: 0.2827 - val_acc: 0.9102
Epoch 64/100
337/337 [==============================] - 0s 80us/sample - loss: 0.2743 - acc: 0.9110 - val_loss: 0.2753 - val_acc: 0.9042
Epoch 65/100
337/337 [==============================] - 0s 80us/sample - loss: 0.2559 - acc: 0.9199 - val_loss: 0.2686 - val_acc: 0.9042
Epoch 66/100
337/337 [==============================] - 0s 80us/sample - loss: 0.2563 - acc: 0.9199 - val_loss: 0.2686 - val_acc: 0.9042
Epoch 67/100
337/337 [==============================] 

In [179]:
# plot tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 11868), started 0:51:34 ago. (Use '!kill 11868' to kill it.)

In [147]:
# Spits out probabilities by default.
model.predict_classes(scaled_X_test)

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [148]:
model.evaluate(x=scaled_X_test, y=y_test)

167/167 [==============================] - 0s 36us/sample - loss: 0.2052 - acc: 0.9222


[0.20515271592996792, 0.9221557]

In [149]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [150]:
predictions = model.predict_classes(scaled_X_test)

In [151]:
predictions

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [152]:
# If you use only predictions it will return scaled values between 0 and 1 due to MinMaxScaler... possible
# workaround below
y_test.argmax(axis=1)

array([0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0], dtype=int64)

In [153]:
print(confusion_matrix(y_test.argmax(axis=1), predictions))

[[90 11]
 [ 2 64]]


In [154]:
print(classification_report(y_test.argmax(axis=1), predictions))

              precision    recall  f1-score   support

           0       0.98      0.89      0.93       101
           1       0.85      0.97      0.91        66

   micro avg       0.92      0.92      0.92       167
   macro avg       0.92      0.93      0.92       167
weighted avg       0.93      0.92      0.92       167



In [155]:
print("Model accuracy: ", accuracy_score(y_test.argmax(axis=1), predictions), "%")

Model accuracy:  0.9221556886227545 %


In [156]:
# Save the Neural Network model architecture
now = datetime.now()
current_time = now.strftime("%d%m%Y_%H%M%S")

# Create logic to create folder structure if it doesn't exist!
model.save(f"train_{current_time}_model.h5")